## This notebook pulls candidate device panel, device panel, and completely home device panel for NYC origin cbgs, and aggregates to borough and NYC total

### Purpose is for sample bias understanding

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
cbg_nyc = pd.read_csv('../data/nyc_cbg.csv')

In [6]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

### Prep county-level table for 2019

Records represent individual CBGs (for later aggregation)

In [7]:
# years, months and days
#years = ['2019','2020']
monthList =["01","02","03","04","05","06", "07","08","09","10","11","12"]
dayNumList =[31, 28, 31, 30, 31, 30, 31, 31,30,31,30,31] 
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

In [8]:
##Check candidate device count for 2019
frames = []
for m in range (0,12):
    for d in range(0, dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/2019/{monthList[m]}/{dayList[d]}/2019-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'2019-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            dff = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")
            dff['date_y-m-d'] = dff['date_range_start'].str[:10]
            frames.append(dff)

2019-01-01
2019-01-02
2019-01-03
2019-01-04
2019-01-05
2019-01-06
2019-01-07
2019-01-08
2019-01-09
2019-01-10
2019-01-11
2019-01-12
2019-01-13
2019-01-14
2019-01-15
2019-01-16
2019-01-17
2019-01-18
2019-01-19
2019-01-20
2019-01-21
2019-01-22
2019-01-23
2019-01-24
2019-01-25
2019-01-26
2019-01-27
2019-01-28
2019-01-29
2019-01-30
2019-01-31
2019-02-01
2019-02-02
2019-02-03
2019-02-04
2019-02-05
2019-02-06
2019-02-07
2019-02-08
2019-02-09
2019-02-10
2019-02-11
2019-02-12
2019-02-13
2019-02-14
2019-02-15
2019-02-16
2019-02-17
2019-02-18
2019-02-19
2019-02-20
2019-02-21
2019-02-22
2019-02-23
2019-02-24
2019-02-25
2019-02-26
2019-02-27
2019-02-28
2019-03-01
2019-03-02
2019-03-03
2019-03-04
2019-03-05
2019-03-06
2019-03-07
2019-03-08
2019-03-09
2019-03-10
2019-03-11
2019-03-12
2019-03-13
2019-03-14
2019-03-15
2019-03-16
2019-03-17
2019-03-18
2019-03-19
2019-03-20
2019-03-21
2019-03-22
2019-03-23
2019-03-24
2019-03-25
2019-03-26
2019-03-27
2019-03-28
2019-03-29
2019-03-30
2019-03-31
2019-04-01

In [9]:
df_19 = pd.concat(frames)
df_19 = df_19[['date_y-m-d','orig_stco','completely_home_device_count','candidate_device_count','device_count']]

### Prep county-level table for 2020

Records represent individual CBGs (for later aggregation)

In [10]:
# updated february day for leap year
monthList =["01","02","03","04","05","06", "07","08"]
dayNumList =[31, 29, 31, 30, 31, 30, 31, 31] 

In [11]:
##Check candidate device count for 2019
frames = []
for m in range (0,8):
    for d in range(0, dayNumList[m]):
        with fs.open(f'sg-c19-response/social-distancing/v2/2020/{monthList[m]}/{dayList[d]}/2020-{monthList[m]}-{dayList[d]}-social-distancing.csv.gz','rb') as f:
            print(f'2020-{monthList[m]}-{dayList[d]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            dff = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")
            dff['date_y-m-d'] = dff['date_range_start'].str[:10]
            frames.append(dff)

2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05
2020-01-06
2020-01-07
2020-01-08
2020-01-09
2020-01-10
2020-01-11
2020-01-12
2020-01-13
2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31

In [12]:
df_20 = pd.concat(frames)
df_20 = df_20[['date_y-m-d','orig_stco','completely_home_device_count','candidate_device_count','device_count']]

In [13]:
df_20.head(20)

,date_y-m-d,orig_stco,completely_home_device_count,candidate_device_count,device_count
0,2020-01-01,36005,1,14,8
1,2020-01-01,36005,25,102,54
2,2020-01-01,36005,42,255,111
3,2020-01-01,36005,21,104,55
4,2020-01-01,36005,2,27,11
5,2020-01-01,36005,17,102,57
6,2020-01-01,36005,17,95,45
7,2020-01-01,36005,56,276,144
8,2020-01-01,36005,3,49,22
9,2020-01-01,36005,55,225,120


### Add 2019 and 2020 tables together

In [14]:
table = pd.concat([df_19,df_20],axis=0)
table.head(20)

,date_y-m-d,orig_stco,completely_home_device_count,candidate_device_count,device_count
0,2019-01-01,36005,2,45,10
1,2019-01-01,36005,41,228,66
2,2019-01-01,36005,69,393,134
3,2019-01-01,36005,47,257,95
4,2019-01-01,36005,8,61,29
5,2019-01-01,36005,40,209,63
6,2019-01-01,36005,40,249,76
7,2019-01-01,36005,117,551,212
8,2019-01-01,36005,11,113,35
9,2019-01-01,36005,104,551,180


In [15]:
#aggregate all of the block-level outputs to county-level
table_co = table.groupby(['date_y-m-d','orig_stco']).agg({'device_count':np.sum,'completely_home_device_count':np.sum,\
                                                    'candidate_device_count':np.sum})

In [16]:
table_co.head(20)

device_count  completely_home_device_count  \
date_y-m-d orig_stco                                               
2019-01-01 36005            109524                         55340   
           36047            165090                         77727   
           36061             86687                         36253   
           36081            148363                         70510   
           36085             36884                         16432   
2019-01-02 36005            109294                         48979   
           36047            165670                         65679   
           36061             87932                         33778   
           36081            148943                         59197   
           36085             36790                         13010   
2019-01-03 36005            108221                         45883   
           36047            163001                         61258   
           36061             86015                         32068   
           36081            147385                         55330   
           36085             36484                         12245   
2019-01-04 36005            109750                         45803   
           36047            164310                         61181   
           36061             86686                         30957   
           36081            148776                         54479   
           36085             36679                         11852   

                      candidate_device_count  
date_y-m-d orig_stco                          
2019-01-01 36005                      370111  
           36047                      512758  
           36061                      317885  
           36081                      454504  
           36085                       94298  
2019-01-02 36005                      370120  
           36047                      512758  
           36061                      317907  
           36081                      454472  
           36085                       94311  
2019-01-03 36005                      370106  
           36047                      512776  
           36061                      317931  
           36081                      454488  
           36085                       94311  
2019-01-04 36005                      370146  
           36047                      512721  
           36061                      317839  
           36081                      454466  
           36085                       94311

In [17]:
#aggregate to NYC total
table_nyc = table.groupby(['date_y-m-d']).agg({'device_count':np.sum,'completely_home_device_count':np.sum,\
                                                    'candidate_device_count':np.sum})
table_nyc.head(20)

,device_count,completely_home_device_count,candidate_device_count
date_y-m-d,,,
2019-01-01,546548,256262,1749556
2019-01-02,548629,220643,1749568
2019-01-03,541106,206784,1749612
2019-01-04,546201,204272,1749483
2019-01-05,548061,255867,1749486
2019-01-06,563047,267085,1749612
2019-01-07,561172,232225,1749621
2019-01-08,565603,230055,1749538
2019-01-09,582798,224450,1749724


In [18]:
table_co.to_excel("output/dara/full_candidate_device_check_borough.xlsx")
table_nyc.to_excel("output/dara/full_candidate_device_check_nyc.xlsx")

In [19]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')

Run time - 21043.251795053482 seconds
